In [1]:
import numpy as np

In [2]:
# Loading the unprocessed data
train_data = np.load('train_big_224_224.npy')
train_target = np.load('train_big_target_224_224.npy')

In [3]:
print(train_data.shape)
print(train_target.shape)

(6464, 224, 224, 3)
(6464, 3)


In [4]:
# Now pre-process the input
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [5]:
train_data = preprocess_input(train_data)

In [6]:
from sklearn.cross_validation import train_test_split
def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

c:\users\flo\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
X_train, X_val, Y_train, Y_val = split_validation_set(train_data, train_target, 0.3)

In [8]:
# Now all other imports
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras.optimizers import RMSprop, SGD
from keras import backend as K

import math

In [9]:
# Zwei helfer:
def printTrainableLayers(model):
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

def printAllLayers(model):
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print('Finished loading!')

Finished loading!


In [11]:
base_model.layers.pop()

In [12]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [42]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 5 classes
predictions = Dense(3, activation='softmax', kernel_initializer='glorot_uniform', name='Predictions')(x)

In [43]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
printAllLayers(model)

0 input_1
1 zero_padding2d_1
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 ac

In [45]:
# set the first 175 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:176]:
    layer.trainable = False

In [46]:
printTrainableLayers(model)

176 dense_5
177 dropout_5
178 dense_6
179 dropout_6
180 Predictions


In [52]:
# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.0003125
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [48]:
filepath="resnet50-improvement-{epoch:02d}-{val_loss:}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
tbCallBack = TensorBoard(log_dir='./graph_resnet50_dropout', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)
earlyStopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=12, verbose=0, mode='auto')
callbacks_list.append(earlyStopping)
lrSched = LearningRateScheduler(step_decay)
callbacks_list.append(lrSched)

In [20]:
nb_train_samples = X_train.shape[0]
nb_val_samples = X_val.shape[0]
batch_size = 64
nb_epochs = 90

In [21]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(X_train, Y_train, batch_size=batch_size)

In [53]:
#opt = RMSprop(lr=1e-3, rho=0.9, decay=0.0)
opt = SGD(lr=0.0003125, momentum=0.9, decay=0.0, nesterov=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [56]:
# serialize model to JSON
model_json = model.to_json()
with open("resnet50_transfer_dropout.json", "w") as json_file:
    json_file.write(model_json)

In [54]:
printTrainableLayers(model)

176 dense_5
177 dropout_5
178 dense_6
179 dropout_6
180 Predictions


In [24]:
#model.load_weights("resnet50-improvement-10-0.921674348749556.hdf5")

In [55]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=nb_epochs,
        validation_data=(X_val, Y_val),
        validation_steps=nb_val_samples // batch_size,
        callbacks=callbacks_list)

Epoch 1/90
16/16 [==============================] - 18s - loss: 0.6172 - categorical_accuracy: 0.7271 - val_loss: 0.9078 - val_categorical_accuracy: 0.6149
Epoch 2/90
16/16 [==============================] - 17s - loss: 0.6750 - categorical_accuracy: 0.6981 - val_loss: 0.8832 - val_categorical_accuracy: 0.6036
Epoch 3/90
16/16 [==============================] - 17s - loss: 0.6462 - categorical_accuracy: 0.7134 - val_loss: 0.8780 - val_categorical_accuracy: 0.6126
Epoch 4/90
16/16 [==============================] - 26s - loss: 0.6438 - categorical_accuracy: 0.7334 - val_loss: 0.9118 - val_categorical_accuracy: 0.6014
Epoch 5/90
16/16 [==============================] - 19s - loss: 0.6748 - categorical_accuracy: 0.7055 - val_loss: 0.8818 - val_categorical_accuracy: 0.5946
Epoch 6/90
16/16 [==============================] - 18s - loss: 0.5152 - categorical_accuracy: 0.7830 - val_loss: 0.9356 - val_categorical_accuracy: 0.5833
Epoch 7/90
16/16 [==============================] - 24s - loss: 

In [26]:
# serialize weights to HDF5
model.save_weights("resnet50_weights.h5")
print("Saved model to disk")

Saved model to disk


# Upload results to kaggle

In [67]:
# load json and create model
json_file = open("resnet50_transfer_dropout.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("resnet50-improvement-10-0.866289990979272.hdf5")

In [65]:
test_data = np.load('test_224_224.npy')

In [66]:
X_test = preprocess_input(test_data)

In [68]:
y_probabilities = model.predict(X_test, batch_size=64)

In [69]:
# Calculate class posteriors probabilities
#y_probabilities = model.predict_generator(test_generator, steps=64)
print(len(y_probabilities))

512


In [70]:
print(y_probabilities[:5])

[[ 0.49333823  0.50496495  0.00169682]
 [ 0.04461051  0.77833569  0.17705378]
 [ 0.15274088  0.72071487  0.12654422]
 [ 0.5491572   0.44593295  0.00490982]
 [ 0.2300469   0.59321475  0.17673837]]


In [72]:
test_data_dir = 'data/test'

In [73]:
# Read Data only for csv generation
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(224, 224),
                                                  batch_size=64,
                                                  shuffle=False)

Found 512 images belonging to 1 classes.


In [82]:
test_generator.filenames[0].split('\\')[1]

'0.jpg'

In [83]:
# Calculate class labels
filenames = [filename.split('\\')[1] for filename in test_generator.filenames]
ids = [filename.split('.')[0] for filename in filenames]

In [87]:
import os, csv

In [88]:
results_path = 'results/'
results_name = 'predictions_resnet50_dropout.csv'
# save results as a csv file in the specified results directory
with open(os.path.join(results_path, results_name), 'w') as file:
    writer = csv.writer(file)
    writer.writerow(('image_name', 'Type_1', 'Type_2', 'Type_3'))
    writer.writerows(zip(filenames, y_probabilities[:, 0], y_probabilities[:, 1], y_probabilities[:, 2]))